# Interacting with LLMs from various providers using Haystack 

### Prompt node initialization - no base instructions

In [ ]:
import os
from dotenv import load_dotenv
import openai
load_dotenv("./../../.env")

# Open AI API Key
openai.api_key = os.getenv("OPENAI_API_KEY")


In [ ]:
from haystack.preview import Pipeline, Document
from haystack.preview.document_stores import InMemoryDocumentStore
from haystack.preview.components.writers import DocumentWriter
from haystack.preview.components.retrievers import InMemoryBM25Retriever, InMemoryEmbeddingRetriever
from haystack.preview.components.embedders import SentenceTransformersTextEmbedder, SentenceTransformersDocumentEmbedder
from haystack.preview.components.generators.openai.gpt import GPTGenerator
from haystack.preview.components.builders.answer_builder import AnswerBuilder
from haystack.preview.components.builders.prompt_builder import PromptBuilder

In [ ]:
prompt_template = """
    Given a question, you provide with a helpful answer using Shakespeare's language.

    \nQuestion: {{question}}
    \nAnswer:
    """


init_pipeline = Pipeline()
init_pipeline.add_component(instance=PromptBuilder(template=prompt_template), name="prompt_builder")
init_pipeline.add_component(instance=GPTGenerator(api_key=os.getenv("OPENAI_API_KEY"),  
                                                  model_name='gpt-4', 
                                                  api_base_url = 'https://api.openai.com/v1',
                                                  ), 
                            name="llm")
init_pipeline.add_component(instance=AnswerBuilder(), name="answer_builder")
init_pipeline.connect("prompt_builder", "llm")
init_pipeline.connect("llm.replies", "answer_builder.replies")
init_pipeline.connect("llm.metadata", "answer_builder.metadata")

questions = ["Who lives in Paris?", "Who lives in Berlin?", "Who lives in Rome?"]


for question in questions:
    result = init_pipeline.run(
        {
            "prompt_builder": {"question": question},
            "answer_builder": {"query": question},
        }
    )

    print(f"Question: {question}")

In [ ]:
questions = ["Who lives in Paris?", "Who lives in Berlin?", "Who lives in Rome?"]


for question in questions:
    result = init_pipeline.run(
        {
            "prompt_builder": {"question": question},
            "answer_builder": {"query": question},
        }
    )

result


Custom component

In [1]:
from scripts.component_to_connect_to_hf import HuggingFaceModelQuery
import os

In [2]:
hugging_face_token_endpoint = os.getenv("mistral_hf_token")
hf_url = "https://je31lgyj1c1qtn2m.us-east-1.aws.endpoints.huggingface.cloud"

mistral = HuggingFaceModelQuery(api_url=hf_url, api_key=hugging_face_token_endpoint)

In [5]:
parameters = {
    "temperature": 0.1,
    "max_length": 500
}

In [6]:
mistral.run("Who lives in Paris?", parameters=parameters)

[{'generated_text': '\n\nThe answer is:\n\nEveryone.\n\nParis is a city of '}]